# LangChain: Evaluation

A etapa de avaliação do modelo é crucial para manter um sistema funcional. É sem dúvidas a parte mais importante da manutenção do modelo.

Futuramente estudar outras técnicas de avaliação de modelos

---

In [13]:
# Lendo API KEY da OpenAI

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm_model = "gpt-4o-mini"

---

### Criando aplicação de perguntas e resposta

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
# Base de dados contendo vários artigos da BBC (retirada do kaggle)
arquivo = "bbc_news_articles.csv"
loader = CSVLoader(file_path=arquivo)
data = loader.load()
# Visualizando alguns arquivos dos que foram carregados
data[:5]

[Document(metadata={'source': 'bbc_news_articles.csv', 'row': 0}, page_content='title: Ukraine: Angry Zelensky vows to punish Russian atrocities\npubDate: Mon, 07 Mar 2022 08:01:56 GMT\nguid: https://www.bbc.co.uk/news/world-europe-60638042\nlink: https://www.bbc.co.uk/news/world-europe-60638042?at_medium=RSS&at_campaign=KARANGA\ndescription: The Ukrainian president says the country will not forgive or forget those who murder its civilians.'),
 Document(metadata={'source': 'bbc_news_articles.csv', 'row': 1}, page_content='title: War in Ukraine: Taking cover in a town under attack\npubDate: Sun, 06 Mar 2022 22:49:58 GMT\nguid: https://www.bbc.co.uk/news/world-europe-60641873\nlink: https://www.bbc.co.uk/news/world-europe-60641873?at_medium=RSS&at_campaign=KARANGA\ndescription: Jeremy Bowen was on the frontline in Irpin, as residents came under Russian fire while trying to flee.'),
 Document(metadata={'source': 'bbc_news_articles.csv', 'row': 2}, page_content="title: Ukraine war 'catastr

In [11]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()

In [12]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

c:\Users\diasd\anaconda3\envs\ChatBot\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
llm = ChatOpenAI(temperature= 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator":"<<<<<>>>>>"
    }
)

In [18]:
query = "Você tem algum arquivo sobre a guerra na ucrânia?"

In [21]:
llm_replacement_model = ChatOpenAI(temperature=0, model="gpt-4o-mini")
response = index.query(query, llm=llm_replacement_model)
response

"Sim, tenho algumas informações sobre a guerra na Ucrânia. Aqui estão alguns artigos relevantes:\n\n1. **War in Ukraine: 'I saw a Russian soldier shoot my father dead in Bucha'** - Este artigo relata o testemunho de um adolescente que descreve atrocidades supostamente cometidas por soldados russos em Bucha. [Leia mais aqui](https://www.bbc.co.uk/news/world-europe-60989121).\n\n2. **Ukraine in maps: Tracking the war with Russia** - Este artigo fornece uma análise sobre o progresso da contra-ofensiva da Ucrânia e a situação nas linhas de frente. [Leia mais aqui](https://www.bbc.co.uk/news/world-europe-60506682).\n\n3. **War in Ukraine: We are holding on, say Mykolaiv residents** - Este artigo fala sobre a resistência e os sentimentos dos moradores de Mykolaiv enquanto a guerra continua. [Leia mais aqui](https://www.bbc.co.uk/news/world-europe-61687586).\n\nSe precisar de mais informações ou detalhes específicos, por favor, me avise!"

In [32]:
query = "Você tem algum artigo sobre a COVID-19?"

In [33]:
llm_replacement_model = ChatOpenAI(temperature=0, model="gpt-4o-mini")
response = index.query(query, llm=llm_replacement_model)
response

'Sim, aqui estão alguns artigos sobre a COVID-19:\n\n1. **Have I got Covid, a bad cold or something else?** - Este artigo discute os novos sintomas da COVID-19 e quando você deve ficar em casa. [Leia mais aqui](https://www.bbc.co.uk/news/health-54145299).\n\n2. **Is the world ready for a great Covid re-opening?** - Este artigo oferece uma visão sobre a vida dois anos após o início da pandemia. [Leia mais aqui](https://www.bbc.co.uk/news/world-us-canada-60758401).\n\n3. **North Korea: The mystery of its Covid outbreak** - Este artigo explora a situação da COVID-19 na Coreia do Norte e o que se sabe sobre a crise. [Leia mais aqui](https://www.bbc.co.uk/news/61651016).\n\n4. **Why Covid is still flooring some people** - Este artigo aborda por que algumas pessoas ainda estão enfrentando sintomas graves da COVID-19. [Leia mais aqui](https://www.bbc.co.uk/news/health-67726685).\n\nSe precisar de mais informações, é só avisar!'

Ou seja, até agora o bot está sendo alimentado pela minha base de dados.

### Evaluation

A ferramenta ``QAGenerateChain`` é usada para gerar perguntas com base em documentos passados para ela, no intuito de testar a capacidade do bot de responder.

- Geração de Perguntas: Dado um texto ou um conjunto de documentos, a QAGenerateChain pode automaticamente gerar perguntas relevantes baseadas no conteúdo do texto.

- Geração de Respostas: Além de gerar perguntas, essa ferramenta também pode gerar as respostas correspondentes, criando pares de pergunta-resposta.

- Avaliação de Modelos: Pode ser usada como parte de um pipeline de avaliação, onde você gera pares de perguntas e respostas para avaliar a performance de um modelo de QA ao responder a essas perguntas.

In [45]:
from langchain.evaluation.qa import QAGenerateChain

In [46]:
# Criando um exemplo de QAGeneretaChain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [47]:
example_gen_chain

QAGenerateChain(prompt=PromptTemplate(input_variables=['doc'], template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000027449C76A50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000027449C52210>, root_client=<openai.OpenAI object at 0x0000027449CF62A0>, root_async_client=<openai.AsyncOpenAI object at 0x0000027449C754C0>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy=''))

Gerando exemplos de perguntas e respostas com base em algumas observações do meu conjunto de dados

In [48]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc":t} for t in data[:5]]
)

c:\Users\diasd\anaconda3\envs\ChatBot\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [53]:
new_examples[4]

{'qa_pairs': {'query': 'What is the main focus of the article titled "Ukraine conflict: Oil price soars to highest level since 2008," published on March 7, 2022?',
  'answer': 'The main focus of the article is the significant increase in oil prices, reaching the highest levels since 2008, and the resulting impact on consumers, particularly through rising fuel prices and higher household bills.'}}

In [54]:
data[5]

Document(metadata={'source': 'bbc_news_articles.csv', 'row': 5}, page_content='title: Ukraine war: PM to hold talks with world leaders on further sanctions\npubDate: Mon, 07 Mar 2022 08:33:29 GMT\nguid: https://www.bbc.co.uk/news/uk-60642926\nlink: https://www.bbc.co.uk/news/uk-60642926?at_medium=RSS&at_campaign=KARANGA\ndescription: Boris Johnson is to meet the Canadian and Dutch PMs, as MPs debate new laws targeting oligarchs.')